In [3]:
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Load datasets
mal_train = pd.read_csv('/content/drive/MyDrive/sarcasm_mal_train.csv')
mal_test = pd.read_csv('/content/drive/MyDrive/sarcasm_mal_test_without_labels.csv')
mal_val = pd.read_csv('/content/drive/MyDrive/sarcasm_mal_dev.csv')


In [6]:
pip install indic-transliteration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 3.0 MB/s eta 0:00:00


In [7]:
pip install --upgrade indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.16.1
    Uninstalling Pygments-2.16.1:
      Successfully uninstalled Pygments-2.16.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
  Attempting uninstall: sphinx
    Found existing installation: Sphinx 5.0.2
    Uninstalling Sphinx-5.0.2:
      Successfully uninstalled Sphinx-5.0.2
ERROR: pip's dependency resolver does not currently 

In [8]:
import pandas as pd
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

def reverse_transliterate_text(text):
    return transliterate(text, sanscript.ITRANS, sanscript.MALAYALAM)

# Apply reverse transliteration to each DataFrame
mal_train['Text'] = mal_train['Text'].apply(reverse_transliterate_text)
mal_val['Text'] = mal_val['Text'].apply(reverse_transliterate_text)
mal_test['Text'] = mal_test['Text'].apply(reverse_transliterate_text)


In [9]:
import re

def remove_emojis(text):
    # Regular expression pattern to match emojis
    emoji_pattern = re.compile(
        "["                        # Match any character in the following ranges
        "\U0001F600-\U0001F64F"    # Emoticons
        "\U0001F300-\U0001F5FF"    # Miscellaneous Symbols and Pictographs
        "\U0001F680-\U0001F6FF"    # Transport and Map Symbols
        "\U0001F700-\U0001F77F"    # Alchemical Symbols
        "\U0001F780-\U0001F7FF"    # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"    # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"    # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"    # Chess Symbols
        "\U0001FA70-\U0001FAFF"    # Symbols and Pictographs Extended-A
        "\U00002702-\U000027B0"    # Dingbats
        "\U000024C2-\U0001F251"    # Enclosed Alphanumeric Supplement
        "]+", flags=re.UNICODE)    # Include Unicode flag for proper handling

    return emoji_pattern.sub(r'', text)


In [10]:
mal_train['Text'] = mal_train['Text'].apply(remove_emojis)
mal_val['Text'] = mal_train['Text'].apply(remove_emojis)
mal_test['Text'] = mal_test['Text'].apply(remove_emojis)

In [11]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import indic_tokenize

In [12]:
malayalam_stopwords = set([
    'അത്', 'എന്റെ', 'ഇത്', 'ഒരു', 'എനിക്ക്', 'ആകും', 'ഇല്ല', 'ഞാൻ', 'താൻ', 'ഉണ്ടാണ്', 'ആക', 'അവിടെ', 'വരിക',
    'അവിടെ', 'കൂടാതെ', 'അദ്ദേഹം', 'അവൾ', 'അതിനാൽ', 'അങ്ങനെ', 'മറ്റു', 'ഒരുപാടു', 'ആണെങ്കിൽ', 'അവർ', 'അവർക്ക്',
    'അവർക്ക്', 'ഇങ്ങനെയാണ്', 'അതുകൂടാതെ', 'എന്തുകൊണ്ടു', 'ഇവിടെ', 'അടുത്ത', 'അവരെ', 'പിന്നീട്', 'അവളുടെ',
    'ആകുന്നു', 'അതുകൂടാതെ', 'എന്താണെന്ന്', 'അവ', 'അവരുടെ', 'ഇവ', 'കഴിഞ്ഞ', 'പിന്നീട്', 'പോലുള്ള', 'എന്തെങ്കിലും',
    'അങ്ങനെയാണ്', 'അതേസമയം', 'അവർക്കു', 'സഹായം', 'തന്നെ', 'വേഗം', 'അവർ', 'നിന്നു', 'അവരെ', 'നിങ്ങളുടെ', 'അവർ',
    'പോലുള്ള', 'ഇപ്പോഴേക്കും', 'ആവശ്യമായ', 'പിറകേ', 'സഹായം', 'അവരെ', 'അവരുടെ', 'അതിനു', 'അല്ല', 'പോലുള്ള',
    'പെരുമാൾ', 'ഇങ്ങനെ', 'ഉണ്ട്', 'അല്ലാത്ത', 'എങ്കിൽ', 'അങ്ങനെയോ', 'പിന്നീട്', 'ഉണ്ടായിരിക്കണം', 'ഉടൻ', 'അത്',
    'മോഡൽ', 'ഇത്ര', 'ആകുന്നു', 'ആകുന്നത്', 'അവിടെ', 'ഇതിനുപരി', 'കൂടുതൽ', 'അക്ക', 'ചെയ്യുന്നു', 'വഴിയുണ്ട്',
    'മറ്റുള്ളവർ', 'ആയിരിക്കും', 'മുമ്പ്', 'ആകും', 'എല്ലാ', 'എന്നാലും', 'എന്തായാലും', 'എന്തായിരിക്കും', 'അതുകൊണ്ടു',
    'ഒരുപാട്', 'അവിടുത്തെ', 'അവിടെ', 'ഇനിയും', 'പിന്നീട്', 'പിന്നീട്', 'ഒടുവിൽ', 'ഇത്', 'അവർക്കു', 'ആവശ്യം'
])


In [13]:
pip install nltk


In [14]:
import re
import nltk
import pandas as pd

# Download the necessary NLTK data (if not already downloaded)
nltk.download('punkt')

def preprocess_text(text):
    # Ensure the text is a string
    if not isinstance(text, str):
        return ''

    # Lowercase the text
    text = text.lower()

    # Keep Malayalam letters and spaces; remove everything else
    text = re.sub(r'[^\u0D00-\u0D7F\s]', ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize text using NLTK
    tokens = nltk.word_tokenize(text)


    # Remove stopwords
    tokens = [word for word in tokens if word not in malayalam_stopwords]

    # Join tokens back into a single string
    return ' '.join(tokens)

# Apply preprocessing to the DataFrames
mal_train['Text'] = mal_train['Text'].apply(preprocess_text)
mal_val['Text'] = mal_val['Text'].apply(preprocess_text)
mal_test['Text'] = mal_test['Text'].apply(preprocess_text)

# Print DataFrames to verify
print("Training DataFrame:")
print(mal_train)
print("\nValidation DataFrame:")
print(mal_val)
print("\nTest DataFrame:")
print(mal_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Training DataFrame:
                                                    Text         labels
0          ഷ്ച്രീന്ശോത് ഏദുക്കന് വന്ന്ഥ് ന്ജന് മാത്രമാണോ      Sarcastic
1      നമ്മുടെ അനു സിത്താര ചേച്ചി വരുന്നത് നോക്കി ഇരു...      Sarcastic
2      ംഓല്ല്യ്ഹൂദ് ഇസ് ഗേത്തിന്ഗ് ബിഗ്ഗേര് അന്ദ് ബിഗ...  Non-sarcastic
3         ഃഓ ആഅ ്ം ംഅമ്മൂക്ക ഇഥു ഓരു പ്വോലി പ്വോലിക്കുമ്  Non-sarcastic
4      ന്ഥാലേ സമ്ഭവമ് പുരാനമ് ആനേലുമ് ബച്ക്ഗ്രോഉന്ദ് ...      Sarcastic
...                                                  ...            ...
13183         ംഅധു ഛ് ണരയനന് പ്രതീക്ഷ ഉള്ള സംവിധയകൻ കൂടി  Non-sarcastic
13184  ക്യ മ ക് ഹൈ ഃഅന്സ് ഹന്സ് ക് ലോത്പോത് ഹോ ജഓഗേ ഛ...  Non-sarcastic
13185  ചന്നേല് ആഅനു ഇശ്തപേത്തല് സുബ്സ്ച്രിബേ ചേയ്യുമോ...  Non-sarcastic
13186                   ണ്തേ പോന്നോ കിദിലമ് മരന വൈതിന്ഗ്  Non-sarcastic
13187  ഈക്കയുദേ ജോഹ്ന്യ് വല്കേര് ഓര്മ്മ വരുന്നൂ വേരേ ...  Non-sarcastic

[13188 rows x 2 columns]

Validation DataFrame:
                                                   Text         lab

In [15]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# For simplicity, concatenate train and validation data
data = pd.concat([mal_train, mal_val], ignore_index=True)

# Vectorize the text using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['Text'])
y = data['labels']  # Assuming 'Label' column exists for target labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a K-Nearest Neighbors model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predict on the test set using KNN
knn_y_pred = knn_model.predict(X_test)

# Evaluate the KNN model
print("KNN Model Accuracy:", accuracy_score(y_test, knn_y_pred))
print("KNN Classification Report:\n", classification_report(y_test, knn_y_pred))

# Predict on tam_test data using KNN
mal_test_vectorized = vectorizer.transform(mal_test['Text'])
mal_test_knn_predictions = knn_model.predict(mal_test_vectorized)

# Print KNN predictions for tam_test data
print("KNN Predictions for tam_test:\n", pd.Series(mal_test_knn_predictions).value_counts())


KNN Model Accuracy: 0.8054948485794567
KNN Classification Report:
                precision    recall  f1-score   support

Non-sarcastic       0.81      0.99      0.89      2581
    Sarcastic       0.49      0.05      0.09       622

     accuracy                           0.81      3203
    macro avg       0.65      0.52      0.49      3203
 weighted avg       0.75      0.81      0.73      3203

KNN Predictions for tam_test:
 Non-sarcastic    2791
Sarcastic          35
Name: count, dtype: int64


In [16]:
KNN_mal_test_predictions_series = pd.Series(mal_test_knn_predictions)

# Get value counts
value_counts = KNN_mal_test_predictions_series.value_counts()

# Print value counts
print("Value counts of predictions:\n", value_counts)

Value counts of predictions:
 Non-sarcastic    2791
Sarcastic          35
Name: count, dtype: int64


In [17]:
results = pd.DataFrame({
    'ID': mal_test['ID'],
    'Label': KNN_mal_test_predictions_series
})

# Print the results
print(results)

# Optionally, save the results to a CSV file
results.to_csv('KNN*mal*test*predictions.csv', index=False)

           ID          Label
0       Id_01  Non-sarcastic
1       Id_02  Non-sarcastic
2       Id_03  Non-sarcastic
3       Id_04  Non-sarcastic
4       Id_05  Non-sarcastic
...       ...            ...
2821  Id_2822  Non-sarcastic
2822  Id_2823  Non-sarcastic
2823  Id_2824  Non-sarcastic
2824  Id_2825  Non-sarcastic
2825  Id_2826  Non-sarcastic

[2826 rows x 2 columns]


In [18]:

# Decision Tree Model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_y_pred = dt_model.predict(X_test)

print("Decision Tree Model Accuracy:", accuracy_score(y_test, dt_y_pred))
print("Decision Tree Classification Report:\n", classification_report(y_test, dt_y_pred))

# Predict on tam_test data using the best performing model (for demonstration, using both models)
mal_test_vectorized = vectorizer.transform(mal_test['Text'])

# Predictions using Decision Tree
dt_mal_test_predictions = dt_model.predict(mal_test_vectorized)
print("Decision Tree Predictions for tam_test data:\n", dt_mal_test_predictions)


Decision Tree Model Accuracy: 0.7489853262566344
Decision Tree Classification Report:
                precision    recall  f1-score   support

Non-sarcastic       0.83      0.87      0.85      2581
    Sarcastic       0.31      0.25      0.28       622

     accuracy                           0.75      3203
    macro avg       0.57      0.56      0.56      3203
 weighted avg       0.73      0.75      0.74      3203

Decision Tree Predictions for tam_test data:
 ['Non-sarcastic' 'Non-sarcastic' 'Non-sarcastic' ... 'Sarcastic'
 'Non-sarcastic' 'Non-sarcastic']


In [19]:
result3 = pd.DataFrame({
    'ID': mal_test['ID'],
    'Label': dt_mal_test_predictions
})

# Print the results
print(result3)

# Optionally, save the results to a CSV file
result3.to_csv('DT*mal*test*predictions.csv', index=False)

           ID          Label
0       Id_01  Non-sarcastic
1       Id_02  Non-sarcastic
2       Id_03  Non-sarcastic
3       Id_04  Non-sarcastic
4       Id_05  Non-sarcastic
...       ...            ...
2821  Id_2822  Non-sarcastic
2822  Id_2823  Non-sarcastic
2823  Id_2824      Sarcastic
2824  Id_2825  Non-sarcastic
2825  Id_2826  Non-sarcastic

[2826 rows x 2 columns]
